In [1]:
import torch

class CustomReLU(torch.nn.Module):
    """
    Custom implementation of the ReLU activation function.
    """
    def __init__(self):
        super(CustomReLU, self).__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Applies the ReLU function element-wise: max(0, x).
        Args:
            x (torch.Tensor): Input tensor
        Returns:
            torch.Tensor: Output tensor with ReLU applied
        """
        return torch.maximum(x, torch.zeros_like(x))

In [2]:
import torch

class ConvolutionalLayer:
    """
    A utility class to create a convolutional layer with a custom ReLU activation.
    """
    @staticmethod
    def create(in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int) -> torch.nn.Sequential:
        """
        Creates a convolutional layer with a custom ReLU activation.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
            kernel_size (int): Size of the convolution kernel
            stride (int): Stride of the convolution
            padding (int): Padding added to the input
        Returns:
            torch.nn.Sequential: A sequential layer containing Conv2d and CustomReLU
        """
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            CustomReLU()
        )

In [3]:
import torch

class FullyConnectedLayer:
    """
    A utility class to create Fully Connected (FC) layers with Xavier initialization.
    """
    @staticmethod
    def Dense(input_dim: int, output_dim: int) -> torch.nn.Linear:
        """
        Creates an FC layer and applies Xavier initialization.
        Args:
            input_dim (int): Number of input features
            output_dim (int): Number of output features
        Returns:
            torch.nn.Linear: Initialized Fully-Connected layer
        """
        layer = torch.nn.Linear(input_dim, output_dim, bias=True)
        torch.nn.init.xavier_uniform_(layer.weight)
        return layer

In [4]:
class CustomVGGNet(torch.nn.Module):
    """
    Customizable VGGNet implementation with architecture selection.
    Supports VGG-A, B, C, D, and E.
    """
    ARCHITECTURES = {
        "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
        "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
        "C": [64, 64, "M", 128, 128, "M", 256, 256, 1, "M", 512, 512, 1, "M", 512, 512, 1, "M"],
        "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
        "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"],
    }

    def __init__(self, architecture="A", num_classes=1000, dropout_rate=0.5):
        """
        Initialize the VGGNet model.
        Args:
            architecture (str): VGG architecture to use (A, B, C, D, E)
            num_classes (int): Number of output classes
            dropout_rate (float): Dropout probability
        """
        super(CustomVGGNet, self).__init__()
        if architecture not in self.ARCHITECTURES:
            raise ValueError(f"Invalid architecture '{architecture}'. Choose from {list(self.ARCHITECTURES.keys())}.")

        self.features = self._make_layers(self.ARCHITECTURES[architecture])
        self.classifier = torch.nn.Sequential(
            FullyConnectedLayer.Dense(7 * 7 * 512, 4096),
            FullyConnectedLayer.Dense(4096, 4096),
            FullyConnectedLayer.Dense(4096, 1000)
        )

    def _make_layers(self, architecture_config):
        """
        Create convolutional layers based on the architecture configuration.
        Args:
            architecture_config (list): List specifying the layers of the architecture
        Returns:
            torch.nn.Sequential: Feature extraction layers
        """
        layers = []
        in_channels = 3
        for layer in architecture_config:
            if layer == "M":
                layers.append(torch.nn.MaxPool2d(kernel_size=2, stride=2))
            elif layer == 1:  # Handle 1x1 convolution for VGG-C
                layers.append(ConvolutionalLayer.create(in_channels, in_channels, 1, 1, 0))
            else:
                layers.append(ConvolutionalLayer.create(in_channels, layer, 1, 1, 0))
                in_channels = layer
        return torch.nn.Sequential(*layers)

    def _calculate_feature_map_size(self, input_size, architecture):
        """
        Calculate the size of the feature map after the convolutional layers.
        Args:
            input_size (int): Size of the input image (assumed square)
            architecture (str): The VGG architecture being used
        Returns:
            int: Feature map size
        """
        size = input_size
        for layer in self.ARCHITECTURES[architecture]:
            if layer == "M":
                size //= 2  # MaxPooling halves the size
            elif layer == 1:
                pass  # 1x1 convolution does not change the size
            else:
                pass  # 3x3 convolution does not change the size
        return size

    def forward(self, x):
        """
        Forward pass of the model.
        Args:
            x (torch.Tensor): Input image tensor
        Returns:
            torch.Tensor: Class scores
        """
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.classifier(x)
        return x

In [5]:
from torchsummary import summary

architecture = input("Choose architecture (A, B, C, D, E): ").upper()
model = CustomVGGNet(architecture=architecture)

print(model.__class__.__name__)
summary(model, input_size=(3, 224, 224))

Choose architecture (A, B, C, D, E):  E


CustomVGGNet
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]             256
        CustomReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]           4,160
        CustomReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]           8,320
        CustomReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          16,512
        CustomReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          33,024
       CustomReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]          65,792
       CustomReLU-14      